In [1]:
from time import time
import pandas as pd
import numpy as np
from PIL import Image
from keras import optimizers
import keras.layers.advanced_activations
import scipy
import random
import matplotlib.pyplot as plt  
from scipy.stats import norm  
import keras
from keras.models import Sequential
from keras.initializers import VarianceScaling,RandomNormal
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, concatenate
from keras.layers import UpSampling2D, Reshape, Lambda, Flatten, Activation,Concatenate
from keras.models import Model  
from keras.optimizers import SGD, Adadelta, Adagrad,Adam
from keras import backend as K  
from keras import objectives  
from keras.utils.vis_utils import plot_model  
from keras.utils import np_utils, generic_utils
import sys 
import tensorflow as tf
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.layers import BatchNormalization
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5'
import sklearn
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
%matplotlib inline
from sklearn import manifold, datasets,cluster
from sklearn.utils import check_random_state
from sklearn import metrics

/home/hsun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
train_img = np.load("img_set_unclip_unlog.npy").reshape([-1,256,256,1])
train_seg = np.load("seg_set_unclip_unlog.npy").reshape([-1,256,256,1])
train_flo = np.load("flo_set_rgb_unclip_unlog.npy")
train_rflo = np.load("rflo_set_rgb_unclip_unlog.npy")
train_img = np.log(train_img+1.0)
train_img = train_img/train_img.max()
train_seg = np_utils.to_categorical(train_seg, 2)

# baseline: without optical flow

In [18]:
img_rows, img_cols = 256, 256
img_channels = 1

batch_size =50
latent_dim = 256
nb_epoch = 50
intermediate_dim =256
original_dim = img_rows*img_cols
LRelu = 'relu'

#USE = 'autoencoder'
#USE = 'vae'
#encoder:

input_img = Input(shape=(img_rows, img_cols,img_channels))

conv_1 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(input_img)
conv_1 = Activation('relu')(conv_1)
conv_1 = BatchNormalization()(conv_1)
maxpool_1 = MaxPooling2D((2, 2),padding='same')(conv_1)
#maxpool_1 = Dropout(0.3)(maxpool_1)


conv_2 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(maxpool_1)
conv_2 = Activation('relu')(conv_2)
#conv_2 = Dropout(0.3)(conv_2)
conv_2 = BatchNormalization()(conv_2)
maxpool_2 = MaxPooling2D((2, 2),  padding='same')(conv_2)
#maxpool_2 = Dropout(0.3)(maxpool_2)

conv_3 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(maxpool_2)
conv_3 = Activation(LRelu)(conv_3)
#conv_3 = Dropout(0.3)(conv_3)
conv_3 = BatchNormalization()(conv_3)
maxpool_3 = MaxPooling2D((2, 2),  padding='same')(conv_3)
#maxpool_3 = Dropout(0.3)(maxpool_3)

conv_4 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(maxpool_3)
conv_4 = Activation(LRelu)(conv_4)
#conv_4 = Dropout(0.3)(conv_4)
conv_4 = BatchNormalization()(conv_4)
maxpool_4 = MaxPooling2D((2, 2),  padding='same')(conv_4)
#maxpool_4 = Dropout(0.3)(maxpool_4)

#conv_5 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(maxpool_4)
#maxpool_5 = MaxPooling2D((2, 2),  padding='same')(conv_5)


#x = Conv2D(5, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(x)
#x = MaxPooling2D((2, 2),  padding='same')(x)

visual = Flatten()(maxpool_4)
h_1 = Dense(intermediate_dim, activation='relu')(visual)#relu?
encoded = Dense(latent_dim, activation='tanh')(h_1)# relu?


h_3 = Dense(intermediate_dim,activation=LRelu)(encoded)#for AE

h_4 = Dense(20*16*16,activation=LRelu)(h_3)
h_5 = Reshape((16,16,20))(h_4)


#conv_6 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(h_5)
#upsample_6 = UpSampling2D((2, 2))(conv_6)

conv_7 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(h_5)
upsample_7 = UpSampling2D((2, 2))(conv_7)

upsample_7 = Concatenate()([upsample_7,conv_4])
conv_8 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(upsample_7)
upsample_8 = UpSampling2D((2, 2))(conv_8)

upsample_8 = Concatenate()([upsample_8,conv_3])
conv_9 = Conv2D(10, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(upsample_8)
upsample_9 = UpSampling2D((2, 2))(conv_9)

upsample_9 = Concatenate()([upsample_9,conv_2])
conv_10 = Conv2D(5,  (3, 3), activation='relu',padding='same',kernel_initializer='normal')(upsample_9)
upsample_10 = UpSampling2D((2, 2))(conv_10)

upsample_10 = Concatenate()([upsample_10,conv_1])
decoded = Conv2D(2, (3, 3), activation='softmax', padding='same')(upsample_10)
#decoded = Flatten()(decoded)
#decoded = Dense(256*256,activation='softmax')(decoded)


EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=0, mode='auto')
checkpoint = ModelCheckpoint('model_keras0516GPU0.h5',monitor ='val_loss',verbose = 1,save_best_only = True)

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true[:,:,:,1]) 
    y_pred_f = K.flatten(y_pred[:,:,:,1]) 
    intersection = K.sum(y_true_f * y_pred_f) 
    return (2. * intersection + 1e-6) / (K.sum(K.square(y_true_f)) + K.sum(K.square(y_pred_f)) + 1e-6)
def dice_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

#def ae_loss(x, decoded):  
#    xent_loss = original_dim * objectives.mean_squared_error(x,decoded)
#    return xent_loss
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
autoencoder = Model(inputs=input_img, outputs=decoded)
autoencoder.compile(optimizer=adam, loss=dice_loss)

In [19]:
#CheckPoint = keras.callbacks.ModelCheckpoint("Model_keras.h5", monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
autoencoder.fit(train_img[:1150], train_seg[:1150],
        shuffle=True,
        epochs=200,
        batch_size=batch_size,
        validation_data=(train_img[1150:],train_seg[1150:]),callbacks=[EarlyStopping,checkpoint])

Train on 1150 samples, validate on 127 samples
Epoch 1/200
1150/1150 [==============================] - 8s 7ms/step - loss: -0.1608 - val_loss: -0.0550

Epoch 00001: val_loss improved from inf to -0.05498, saving model to model_keras0516GPU0.h5
Epoch 2/200
1150/1150 [==============================] - 6s 5ms/step - loss: -0.2217 - val_loss: -0.0856

Epoch 00002: val_loss improved from -0.05498 to -0.08559, saving model to model_keras0516GPU0.h5
Epoch 3/200
1150/1150 [==============================] - 6s 5ms/step - loss: -0.3306 - val_loss: -0.1789

Epoch 00003: val_loss improved from -0.08559 to -0.17892, saving model to model_keras0516GPU0.h5
Epoch 4/200
1150/1150 [==============================] - 6s 5ms/step - loss: -0.5262 - val_loss: -0.2790

Epoch 00004: val_loss improved from -0.17892 to -0.27899, saving model to model_keras0516GPU0.h5
Epoch 5/200
1150/1150 [==============================] - 6s 5ms/step - loss: -0.5868 - val_loss: -0.2036

Epoch 00005: val_loss did not improve fr

# then use optical flow, concate 到最后一层

In [21]:
img_rows, img_cols = 256, 256
img_channels = 1

batch_size =50
latent_dim = 256
nb_epoch = 50
intermediate_dim =256
original_dim = img_rows*img_cols
LRelu = 'relu'

#USE = 'autoencoder'
#USE = 'vae'
#encoder:

input_img = Input(shape=(img_rows, img_cols,img_channels))
input_flo = Input(shape=(img_rows, img_cols,3))
input_rflo = Input(shape=(img_rows, img_cols,3))

conv_1 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(input_img)
conv_1 = Activation('relu')(conv_1)
conv_1 = BatchNormalization()(conv_1)
maxpool_1 = MaxPooling2D((2, 2),padding='same')(conv_1)
#maxpool_1 = Dropout(0.3)(maxpool_1)


conv_2 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(maxpool_1)
conv_2 = Activation('relu')(conv_2)
#conv_2 = Dropout(0.3)(conv_2)
conv_2 = BatchNormalization()(conv_2)
maxpool_2 = MaxPooling2D((2, 2),  padding='same')(conv_2)
#maxpool_2 = Dropout(0.3)(maxpool_2)

conv_3 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(maxpool_2)
conv_3 = Activation(LRelu)(conv_3)
#conv_3 = Dropout(0.3)(conv_3)
conv_3 = BatchNormalization()(conv_3)
maxpool_3 = MaxPooling2D((2, 2),  padding='same')(conv_3)
#maxpool_3 = Dropout(0.3)(maxpool_3)

conv_4 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(maxpool_3)
conv_4 = Activation(LRelu)(conv_4)
#conv_4 = Dropout(0.3)(conv_4)
conv_4 = BatchNormalization()(conv_4)
maxpool_4 = MaxPooling2D((2, 2),  padding='same')(conv_4)
#maxpool_4 = Dropout(0.3)(maxpool_4)

#conv_5 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(maxpool_4)
#maxpool_5 = MaxPooling2D((2, 2),  padding='same')(conv_5)


#x = Conv2D(5, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(x)
#x = MaxPooling2D((2, 2),  padding='same')(x)

visual = Flatten()(maxpool_4)
h_1 = Dense(intermediate_dim, activation='relu')(visual)#relu?
encoded = Dense(latent_dim, activation='tanh')(h_1)# relu?


h_3 = Dense(intermediate_dim,activation=LRelu)(encoded)#for AE

h_4 = Dense(20*16*16,activation=LRelu)(h_3)
h_5 = Reshape((16,16,20))(h_4)


#conv_6 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(h_5)
#upsample_6 = UpSampling2D((2, 2))(conv_6)

conv_7 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(h_5)
upsample_7 = UpSampling2D((2, 2))(conv_7)

upsample_7 = Concatenate()([upsample_7,conv_4])
conv_8 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(upsample_7)
upsample_8 = UpSampling2D((2, 2))(conv_8)

upsample_8 = Concatenate()([upsample_8,conv_3])
conv_9 = Conv2D(10, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(upsample_8)
upsample_9 = UpSampling2D((2, 2))(conv_9)

upsample_9 = Concatenate()([upsample_9,conv_2])
conv_10 = Conv2D(5,  (3, 3), activation='relu',padding='same',kernel_initializer='normal')(upsample_9)
upsample_10 = UpSampling2D((2, 2))(conv_10)

upsample_10 = Concatenate()([upsample_10,conv_1,input_flo,input_rflo])
decoded = Conv2D(2, (3, 3), activation='softmax', padding='same')(upsample_10)
#decoded = Flatten()(decoded)
#decoded = Dense(256*256,activation='softmax')(decoded)


EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=0, mode='auto')
checkpoint = ModelCheckpoint('model_keras0516GPU0.h5',monitor ='val_loss',verbose = 1,save_best_only = True)

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true[:,:,:,1]) 
    y_pred_f = K.flatten(y_pred[:,:,:,1]) 
    intersection = K.sum(y_true_f * y_pred_f) 
    return (2. * intersection + 1e-6) / (K.sum(K.square(y_true_f)) + K.sum(K.square(y_pred_f)) + 1e-6)
def dice_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

#def ae_loss(x, decoded):  
#    xent_loss = original_dim * objectives.mean_squared_error(x,decoded)
#    return xent_loss
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
autoencoder = Model(inputs=[input_img,input_flo,input_rflo], outputs=decoded)
autoencoder.compile(optimizer=adam, loss=dice_loss)

In [23]:
#CheckPoint = keras.callbacks.ModelCheckpoint("Model_keras.h5", monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
autoencoder.fit([train_img[:1150],train_flo[:1150],train_rflo[:1150]], train_seg[:1150],
        shuffle=True,
        epochs=200,
        batch_size=batch_size,
        validation_data=([train_img[1150:],train_flo[1150:],train_rflo[1150:]],train_seg[1150:]),callbacks=[EarlyStopping,checkpoint])

Train on 1150 samples, validate on 127 samples
Epoch 1/200
1150/1150 [==============================] - 11s 10ms/step - loss: -0.1514 - val_loss: -0.0521

Epoch 00001: val_loss improved from inf to -0.05208, saving model to model_keras0516GPU0.h5
Epoch 2/200
1150/1150 [==============================] - 8s 7ms/step - loss: -0.1654 - val_loss: -0.0571

Epoch 00002: val_loss improved from -0.05208 to -0.05706, saving model to model_keras0516GPU0.h5
Epoch 3/200
1150/1150 [==============================] - 7s 6ms/step - loss: -0.1841 - val_loss: -0.0663

Epoch 00003: val_loss improved from -0.05706 to -0.06634, saving model to model_keras0516GPU0.h5
Epoch 4/200
1150/1150 [==============================] - 7s 6ms/step - loss: -0.2183 - val_loss: -0.0993

Epoch 00004: val_loss improved from -0.06634 to -0.09927, saving model to model_keras0516GPU0.h5
Epoch 5/200
1150/1150 [==============================] - 7s 6ms/step - loss: -0.3215 - val_loss: -0.3509

Epoch 00005: val_loss improved from -0

# mid layer concate

In [24]:
img_rows, img_cols = 256, 256
img_channels = 1

batch_size =50
latent_dim = 256
nb_epoch = 50
intermediate_dim =256
original_dim = img_rows*img_cols
LRelu = 'relu'

#USE = 'autoencoder'
#USE = 'vae'
#encoder:

input_img = Input(shape=(img_rows, img_cols,img_channels))
input_flo = Input(shape=(img_rows, img_cols,3))
input_rflo = Input(shape=(img_rows, img_cols,3))

concate_flo = Concatenate()([input_flo,input_rflo])
conv_flo_1 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(concate_flo)
conv_flo_1 = Activation('relu')(conv_flo_1)
conv_flo_1 = BatchNormalization()(conv_flo_1)
maxpool_flo_1 = MaxPooling2D((2, 2),padding='same')(conv_flo_1)
conv_flo_2 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(maxpool_flo_1)
conv_flo_2 = Activation('relu')(conv_flo_2)
conv_flo_2 = BatchNormalization()(conv_flo_2)
maxpool_flo_2 = MaxPooling2D((2, 2),  padding='same')(conv_flo_2)
conv_flo_3 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(maxpool_flo_2)
conv_flo_3 = Activation(LRelu)(conv_flo_3)
conv_flo_3 = BatchNormalization()(conv_flo_3)
maxpool_flo_3 = MaxPooling2D((2, 2),  padding='same')(conv_flo_3)
conv_flo_4 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(maxpool_flo_3)
conv_flo_4 = Activation(LRelu)(conv_flo_4)
conv_flo_4 = BatchNormalization()(conv_flo_4)
maxpool_flo_4 = MaxPooling2D((2, 2),  padding='same')(conv_flo_4)






conv_1 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(input_img)
conv_1 = Activation('relu')(conv_1)
conv_1 = BatchNormalization()(conv_1)
maxpool_1 = MaxPooling2D((2, 2),padding='same')(conv_1)
#maxpool_1 = Dropout(0.3)(maxpool_1)

conv_2 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(maxpool_1)
conv_2 = Activation('relu')(conv_2)
#conv_2 = Dropout(0.3)(conv_2)
conv_2 = BatchNormalization()(conv_2)
maxpool_2 = MaxPooling2D((2, 2),  padding='same')(conv_2)
#maxpool_2 = Dropout(0.3)(maxpool_2)

conv_3 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(maxpool_2)
conv_3 = Activation(LRelu)(conv_3)
#conv_3 = Dropout(0.3)(conv_3)
conv_3 = BatchNormalization()(conv_3)
maxpool_3 = MaxPooling2D((2, 2),  padding='same')(conv_3)
#maxpool_3 = Dropout(0.3)(maxpool_3)

conv_4 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(maxpool_3)
conv_4 = Activation(LRelu)(conv_4)
#conv_4 = Dropout(0.3)(conv_4)
conv_4 = BatchNormalization()(conv_4)
maxpool_4 = MaxPooling2D((2, 2),  padding='same')(conv_4)
#maxpool_4 = Dropout(0.3)(maxpool_4)

#conv_5 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(maxpool_4)
#maxpool_5 = MaxPooling2D((2, 2),  padding='same')(conv_5)


#x = Conv2D(5, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(x)
#x = MaxPooling2D((2, 2),  padding='same')(x)

maxpool_4 = Concatenate()([maxpool_flo_4,maxpool_4])
visual = Flatten()(maxpool_4)
h_1 = Dense(intermediate_dim, activation='relu')(visual)#relu?
encoded = Dense(latent_dim, activation='tanh')(h_1)# relu?


h_3 = Dense(intermediate_dim,activation=LRelu)(encoded)#for AE

h_4 = Dense(20*16*16,activation=LRelu)(h_3)
h_5 = Reshape((16,16,20))(h_4)


#conv_6 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(h_5)
#upsample_6 = UpSampling2D((2, 2))(conv_6)

conv_7 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(h_5)
upsample_7 = UpSampling2D((2, 2))(conv_7)

upsample_7 = Concatenate()([upsample_7,conv_4])
conv_8 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(upsample_7)
upsample_8 = UpSampling2D((2, 2))(conv_8)

upsample_8 = Concatenate()([upsample_8,conv_3])
conv_9 = Conv2D(10, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(upsample_8)
upsample_9 = UpSampling2D((2, 2))(conv_9)

upsample_9 = Concatenate()([upsample_9,conv_2])
conv_10 = Conv2D(5,  (3, 3), activation='relu',padding='same',kernel_initializer='normal')(upsample_9)
upsample_10 = UpSampling2D((2, 2))(conv_10)

upsample_10 = Concatenate()([upsample_10,conv_1,input_flo,input_rflo])
decoded = Conv2D(2, (3, 3), activation='softmax', padding='same')(upsample_10)
#decoded = Flatten()(decoded)
#decoded = Dense(256*256,activation='softmax')(decoded)


EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=0, mode='auto')
checkpoint = ModelCheckpoint('model_keras0516GPU0.h5',monitor ='val_loss',verbose = 1,save_best_only = True)

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true[:,:,:,1]) 
    y_pred_f = K.flatten(y_pred[:,:,:,1]) 
    intersection = K.sum(y_true_f * y_pred_f) 
    return (2. * intersection + 1e-6) / (K.sum(K.square(y_true_f)) + K.sum(K.square(y_pred_f)) + 1e-6)
def dice_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

#def ae_loss(x, decoded):  
#    xent_loss = original_dim * objectives.mean_squared_error(x,decoded)
#    return xent_loss
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
autoencoder = Model(inputs=[input_img,input_flo,input_rflo], outputs=decoded)
autoencoder.compile(optimizer=adam, loss=dice_loss)

In [25]:
#CheckPoint = keras.callbacks.ModelCheckpoint("Model_keras.h5", monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
autoencoder.fit([train_img[:1150],train_flo[:1150],train_rflo[:1150]], train_seg[:1150],
        shuffle=True,
        epochs=200,
        batch_size=batch_size,
        validation_data=([train_img[1150:],train_flo[1150:],train_rflo[1150:]],train_seg[1150:]),callbacks=[EarlyStopping,checkpoint])

Train on 1150 samples, validate on 127 samples
Epoch 1/200
1150/1150 [==============================] - 13s 11ms/step - loss: -0.2390 - val_loss: -0.1260

Epoch 00001: val_loss improved from inf to -0.12595, saving model to model_keras0516GPU0.h5
Epoch 2/200
1150/1150 [==============================] - 9s 8ms/step - loss: -0.2941 - val_loss: -0.2423

Epoch 00002: val_loss improved from -0.12595 to -0.24228, saving model to model_keras0516GPU0.h5
Epoch 3/200
1150/1150 [==============================] - 10s 9ms/step - loss: -0.4342 - val_loss: -0.4294

Epoch 00003: val_loss improved from -0.24228 to -0.42942, saving model to model_keras0516GPU0.h5
Epoch 4/200
1150/1150 [==============================] - 11s 9ms/step - loss: -0.5449 - val_loss: -0.4567

Epoch 00004: val_loss improved from -0.42942 to -0.45674, saving model to model_keras0516GPU0.h5
Epoch 5/200
1150/1150 [==============================] - 9s 8ms/step - loss: -0.6201 - val_loss: -0.5090

Epoch 00005: val_loss improved from 

# conv to last lyr

In [28]:
img_rows, img_cols = 256, 256
img_channels = 1

batch_size =50
latent_dim = 256
nb_epoch = 50
intermediate_dim =256
original_dim = img_rows*img_cols
LRelu = 'relu'

#USE = 'autoencoder'
#USE = 'vae'
#encoder:

input_img = Input(shape=(img_rows, img_cols,img_channels))
input_flo = Input(shape=(img_rows, img_cols,3))
input_rflo = Input(shape=(img_rows, img_cols,3))

concate_flo = Concatenate()([input_flo,input_rflo])
conv_flo_1 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(concate_flo)
conv_flo_1 = Activation('relu')(conv_flo_1)
conv_flo_1 = BatchNormalization()(conv_flo_1)
#maxpool_flo_1 = MaxPooling2D((2, 2),padding='same')(conv_flo_1)
conv_flo_2 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(conv_flo_1)
conv_flo_2 = Activation('relu')(conv_flo_2)
conv_flo_2 = BatchNormalization()(conv_flo_2)
#maxpool_flo_2 = MaxPooling2D((2, 2),  padding='same')(conv_flo_2)
conv_flo_3 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(conv_flo_2)
conv_flo_3 = Activation(LRelu)(conv_flo_3)
conv_flo_3 = BatchNormalization()(conv_flo_3)






conv_1 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(input_img)
conv_1 = Activation('relu')(conv_1)
conv_1 = BatchNormalization()(conv_1)
maxpool_1 = MaxPooling2D((2, 2),padding='same')(conv_1)
#maxpool_1 = Dropout(0.3)(maxpool_1)

conv_2 = Conv2D(20, (3, 3), padding='same',kernel_initializer='normal',dilation_rate=2)(maxpool_1)
conv_2 = Activation('relu')(conv_2)
#conv_2 = Dropout(0.3)(conv_2)
conv_2 = BatchNormalization()(conv_2)
maxpool_2 = MaxPooling2D((2, 2),  padding='same')(conv_2)
#maxpool_2 = Dropout(0.3)(maxpool_2)

conv_3 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(maxpool_2)
conv_3 = Activation(LRelu)(conv_3)
#conv_3 = Dropout(0.3)(conv_3)
conv_3 = BatchNormalization()(conv_3)
maxpool_3 = MaxPooling2D((2, 2),  padding='same')(conv_3)
#maxpool_3 = Dropout(0.3)(maxpool_3)

conv_4 = Conv2D(20, (3, 3),padding='same',kernel_initializer='normal',dilation_rate=1)(maxpool_3)
conv_4 = Activation(LRelu)(conv_4)
#conv_4 = Dropout(0.3)(conv_4)
conv_4 = BatchNormalization()(conv_4)
maxpool_4 = MaxPooling2D((2, 2),  padding='same')(conv_4)
#maxpool_4 = Dropout(0.3)(maxpool_4)

#conv_5 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(maxpool_4)
#maxpool_5 = MaxPooling2D((2, 2),  padding='same')(conv_5)


#x = Conv2D(5, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(x)
#x = MaxPooling2D((2, 2),  padding='same')(x)


visual = Flatten()(maxpool_4)
h_1 = Dense(intermediate_dim, activation='relu')(visual)#relu?
encoded = Dense(latent_dim, activation='tanh')(h_1)# relu?


h_3 = Dense(intermediate_dim,activation=LRelu)(encoded)#for AE

h_4 = Dense(20*16*16,activation=LRelu)(h_3)
h_5 = Reshape((16,16,20))(h_4)


#conv_6 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(h_5)
#upsample_6 = UpSampling2D((2, 2))(conv_6)

conv_7 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(h_5)
upsample_7 = UpSampling2D((2, 2))(conv_7)

upsample_7 = Concatenate()([upsample_7,conv_4])
conv_8 = Conv2D(20, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(upsample_7)
upsample_8 = UpSampling2D((2, 2))(conv_8)

upsample_8 = Concatenate()([upsample_8,conv_3])
conv_9 = Conv2D(10, (3, 3), activation='relu', padding='same',kernel_initializer='normal')(upsample_8)
upsample_9 = UpSampling2D((2, 2))(conv_9)

upsample_9 = Concatenate()([upsample_9,conv_2])
conv_10 = Conv2D(5,  (3, 3), activation='relu',padding='same',kernel_initializer='normal')(upsample_9)
upsample_10 = UpSampling2D((2, 2))(conv_10)

upsample_10 = Concatenate()([upsample_10,conv_1,conv_flo_3])
decoded = Conv2D(2, (3, 3), activation='softmax', padding='same')(upsample_10)
#decoded = Flatten()(decoded)
#decoded = Dense(256*256,activation='softmax')(decoded)


EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=0, mode='auto')
checkpoint = ModelCheckpoint('model_keras0516GPU0.h5',monitor ='val_loss',verbose = 1,save_best_only = True)

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true[:,:,:,1]) 
    y_pred_f = K.flatten(y_pred[:,:,:,1]) 
    intersection = K.sum(y_true_f * y_pred_f) 
    return (2. * intersection + 1e-6) / (K.sum(K.square(y_true_f)) + K.sum(K.square(y_pred_f)) + 1e-6)
def dice_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

#def ae_loss(x, decoded):  
#    xent_loss = original_dim * objectives.mean_squared_error(x,decoded)
#    return xent_loss
adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
autoencoder = Model(inputs=[input_img,input_flo,input_rflo], outputs=decoded)
autoencoder.compile(optimizer=adam, loss=dice_loss)

In [29]:
#CheckPoint = keras.callbacks.ModelCheckpoint("Model_keras.h5", monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
autoencoder.fit([train_img[:1150],train_flo[:1150],train_rflo[:1150]], train_seg[:1150],
        shuffle=True,
        epochs=200,
        batch_size=batch_size,
        validation_data=([train_img[1150:],train_flo[1150:],train_rflo[1150:]],train_seg[1150:]),callbacks=[EarlyStopping,checkpoint])

Train on 1150 samples, validate on 127 samples
Epoch 1/200
1150/1150 [==============================] - 17s 15ms/step - loss: -0.2013 - val_loss: -0.0755

Epoch 00001: val_loss improved from inf to -0.07548, saving model to model_keras0516GPU0.h5
Epoch 2/200
1150/1150 [==============================] - 13s 11ms/step - loss: -0.2368 - val_loss: -0.1041

Epoch 00002: val_loss improved from -0.07548 to -0.10408, saving model to model_keras0516GPU0.h5


OSError: Unable to create file (unable to lock file, errno = 11, error message = 'Resource temporarily unavailable')